In [13]:
import torch

In [14]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

if device.type=='cuda':
    print(torch.cuda.get_device_name(0))

cpu


In [15]:
CLASSES=10
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [16]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
#import matplotlib.pyplot as plt
import numpy as np

In [17]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=96,kernel_size=11,stride=4,padding=0) # ((224 - 11 + 2 * 0) / 4) + 1 = 55
        self.conv2 = nn.Conv2d(in_channels=96,out_channels=96,kernel_size=1) # ((55 - 1 + 2 * 0) / 1) + 1 = 55
        self.conv3=nn.Conv2d(in_channels=96,out_channels=96,kernel_size=1) # ((55 - 1 + 2 * 0) / 1) + 1 = 55
        self.pool3=nn.MaxPool2d(kernel_size=3,stride=2) # ((55 - 3 + 2 * 0) / 2) + 1 = 27
        self.conv4=nn.Conv2d(in_channels=96,out_channels=256,kernel_size=11,stride=4,padding=2) # ((27 - 11 + 2 * 1) / 4) + 1 = 5
        self.conv5a=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=1) # ((5 - 1 + 2 * 0) / 1) + 1 = 5
        self.conv5b=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=1) # ((5 - 1 + 2 * 0) / 1) + 1 = 5
        self.pool5=nn.MaxPool2d(kernel_size=3,stride=2) # ((5 - 3 + 2 * 0) / 2) + 1 = 2
        self.conv6=nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,stride=1,padding=1) # ((2 - 3 + 2 * 1) / 1) + 1 = 2
        self.conv7=nn.Conv2d(in_channels=384,out_channels=384,kernel_size=1) # ((2 - 1 + 2 * 0) / 1) + 1 = 2
        self.conv8=nn.Conv2d(in_channels=384,out_channels=384,kernel_size=1) # ((2 - 1 + 2 * 0) / 1) + 1 = 2
        self.fc9 = nn.Linear(in_features=(2*2*384),out_features=4096)
        self.fc10 = nn.Linear(in_features=4096,out_features=4096)
        self.fc11 = nn.Linear(in_features=4096,out_features=CLASSES)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.dropout(self.pool3(F.relu(self.conv3(x))), 0.5)
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5a(x))
        x = F.dropout(self.pool5(F.relu(self.conv5b(x))), 0.5)
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.dropout(F.relu(self.conv8(x)), 0.5)
        x = torch.flatten(x, 1)
        x = F.dropout(F.relu(self.fc9(x)), 0.5)
        x = F.dropout(F.relu(self.fc10(x)), 0.5)
        x = self.fc11(x)
        
        return x

In [18]:
transform_conf=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [19]:
BATCH_SIZE=16
train_dataset=datasets.MNIST('~/data/',train=True,download=True,transform=transform_conf)
test_dataset=datasets.MNIST('~/data/',train=False,download=True,transform=transform_conf)

In [20]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=BATCH_SIZE,shuffle=True)

In [21]:
model=Net().to(device)
optimizer=optim.Adam(params=model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [22]:
def train(model,device,train_loader,optimizer,epochs):
    print("inside train")
    model.train()
    for batch_ids, (img, classes) in enumerate(train_loader):
        classes=classes.type(torch.LongTensor)
        img,classes=img.to(device),classes.to(device)
        torch.autograd.set_detect_anomaly(True)
        optimizer.zero_grad()
        output=model(img)
        loss = loss_fn(output,classes)
        
        loss.backward()
        optimizer.step()
    if(batch_ids +1) % 2 == 0:
        print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epochs, batch_ids* len(img), len(train_loader.dataset),
            100.*batch_ids / len(train_loader),loss.item()))

In [23]:
def test(model, device, test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for img,classes in test_loader:
            img,classes=img.to(device), classes.to(device)
            y_hat=model(img)
            test_loss+=F.nll_loss(y_hat,classes,reduction='sum').item()
            _,y_pred=torch.max(y_hat,1)
            correct+=(y_pred==classes).sum().item()
        test_loss/=len(test_dataset)
        print("\n Test set: Avarage loss: {:.0f},Accuracy:{}/{} ({:.0f}%)\n".format(
            test_loss,correct,len(test_dataset),100.*correct/len(test_dataset)))
        print('='*30)

In [24]:
if __name__=='__main__':
    seed=42
    EPOCHS=1
    
    for epoch in range(1,EPOCHS+1):
        train(model,device,train_loader,optimizer,epoch)
        test(model,device,test_loader)

inside train


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x1536 and 64896x4096)